# 进行大模型加载 本地知识库构建和训练

In [14]:
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# 初始化本地模型
llm = Ollama(model="qwen2.5:7b-instruct", temperature=0.5)





In [3]:
template_messages = [
    SystemMessage(content="""
现在你是一个意图判断助手，你需要根据用户的提问判断用户的意图，并以字典返回。
简单来说，你现在需要做出二分类判断并直接输出简单的判断结果，如true和false
当用户提问需要进行菜谱推荐时，或回答与菜谱知识想关的问题时，回答true
当用户的提问你不需要借助菜谱知识时，回答false
                  
如'用户提问：我有牛肉和土豆，可以做什么菜？ 你需要回答：true'
如'用户提问：今天天气怎么样？ 你需要回答：false'
    """),
    HumanMessage(content="用户提问：{question}")
]


In [10]:
query='哈喽'

In [11]:
formatted_messages = []
for msg in template_messages:
    formatted_content = msg.content.format(question=query)
    if isinstance(msg, SystemMessage):
        formatted_messages.append(SystemMessage(content=formatted_content))
    elif isinstance(msg, HumanMessage):
        formatted_messages.append(HumanMessage(content=formatted_content))

# 拼接成完整的 prompt 字符串（也可以直接传递消息列表给部分链）
full_prompt = "\n".join([msg.content for msg in formatted_messages])
full_prompt

"\n现在你是一个意图判断助手，你需要根据用户的提问判断用户的意图，并以字典返回。\n简单来说，你现在需要做出二分类判断并直接输出简单的判断结果，如true和false\n当用户提问需要进行菜谱推荐时，或回答与菜谱知识想关的问题时，回答true\n当用户的提问你不需要借助菜谱知识时，回答false\n                  \n如'用户提问：我有牛肉和土豆，可以做什么菜？ 你需要回答：true'\n如'用户提问：今天天气怎么样？ 你需要回答：false'\n    \n用户提问：哈喽"

In [12]:
# 调用对话链生成回答
for chunk in llm.stream(full_prompt):
    print(chunk, end="", flush=True)

false

In [13]:
res=llm.invoke(full_prompt)
if res=='false':
    print(1)
    

1


In [15]:
template_messages = [
    SystemMessage(content="""你是一个做菜小助手，名字叫智小厨，可以回答用户的问题。
    - 你需要基于以下结构化菜谱知识推荐合适的菜谱，并给出推荐理由。
    菜谱知识：{query}
    你可以润色知识库的内容，但不能改变其含义。
    你的回答需要以 JSON 格式字符串输出，包含以下字段：
        - "recipe": 菜谱数据（从知识库中提取出菜品id返回）
        - "reason": 推荐理由（字符串）
    """),
    HumanMessage(content="用户提问：{question}")
]


In [16]:
# 从知识库检索
import RAG
rag = RAG.RAG()
vector_store = rag.load_vector_store()




正在加载向量数据库从: ./chroma_db


d:\NLP\DPR1\RAG.py:136: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=embedding_model)
d:\NLP\DPR1\RAG.py:137: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_store = Chroma(


向量数据库加载成功


In [17]:
# 用户问题
question = "土豆烧牛肉怎么做？"
query_old = rag.retrieve(vector_store, question, 2)
print(query_old)
query = "\n".join([doc.page_content for doc in query_old]) if query_old else "暂无相关菜谱知识"
query

d:\NLP\DPR1\RAG.py:165: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


[Document(metadata={'source': 'knowledge_base_docs\\docx.docs\\output.docx', 'start_index': 485527}, page_content="'在炖肉的同时，把土豆洗净去皮，切成滚刀块。胡萝卜切1cm左右的小丁。', '小葱洗净切成末，大蒜去皮切片。', '牛肉炖好后会发现肉汤上飘着一大层黑乎乎的沫子，这些都是牛肉里的脏东西。', '把牛肉一块块夹出，沥干水分置于小碗中。', '炒锅内倒少许油，待油热后放入蒜片炸出香味。', '倒入炖好的牛肉不停翻炒至肉香味飘出。', '倒入土豆块，继续不停翻炒至土豆略有粘锅。', '倒入没过食物的清水，加入盐、生抽、耗油、少许糖、料酒和老抽，并加入剩余的小米椒（不要掰段），倒入胡萝卜，转中火炖约20分钟。', '转大火，加入少许小葱末稍微煸炒几下，收汁出锅。', '如此美味，你抗拒的了吗？']"), Document(metadata={'source': 'knowledge_base_docs\\docx.docs\\output.docx', 'start_index': 2087309}, page_content="step: ['锅里烧水，水开后转为中火，倒入几滴食用油，将切好的牛肉块倒入锅里并倒入适量香米酒，飞水约七分钟，然后用热水将飞过水的牛肉洗净，提醒，一定要热水洗净，晾干，放入盆里，倒入适量的油和少量老抽腌制半小时，然后锅里倒适量油，中火将牛肉几面煎黄，如图。然后盛在盘子里。', '在等待腌制牛肉的时候，拍好大蒜和老姜，切好泰国绿辣椒', '将辅料准备好后，锅里倒油，花椒冷油下锅，小火熬五分钟，然后放入郫县豆瓣酱和干红辣椒，小火熬五分钟', '然后将八角，桂皮，小茴香，姜蒜倒入锅里，小火八分钟，再转中火四分钟，闻着香味出来了，转大火两分钟熬', '大火两分钟满后倒入牛肉，中火继续翻炒三分钟，加入没过牛肉的开水，记住开水下锅，牛肉不会柴。盖好锅盖，中火炖一个小时二十分钟，中间要开锅盖翻炒下牛肉，避免粘锅。', '然后将土豆去皮切中块，用蒸锅蒸十三分钟，用筷子插一下土豆基本可以插开就好，不用特别熟，一会儿后面还要继续下锅和牛肉一起烧', '将洋葱和葱切好备用，如图',")]


"'在炖肉的同时，把土豆洗净去皮，切成滚刀块。胡萝卜切1cm左右的小丁。', '小葱洗净切成末，大蒜去皮切片。', '牛肉炖好后会发现肉汤上飘着一大层黑乎乎的沫子，这些都是牛肉里的脏东西。', '把牛肉一块块夹出，沥干水分置于小碗中。', '炒锅内倒少许油，待油热后放入蒜片炸出香味。', '倒入炖好的牛肉不停翻炒至肉香味飘出。', '倒入土豆块，继续不停翻炒至土豆略有粘锅。', '倒入没过食物的清水，加入盐、生抽、耗油、少许糖、料酒和老抽，并加入剩余的小米椒（不要掰段），倒入胡萝卜，转中火炖约20分钟。', '转大火，加入少许小葱末稍微煸炒几下，收汁出锅。', '如此美味，你抗拒的了吗？']\nstep: ['锅里烧水，水开后转为中火，倒入几滴食用油，将切好的牛肉块倒入锅里并倒入适量香米酒，飞水约七分钟，然后用热水将飞过水的牛肉洗净，提醒，一定要热水洗净，晾干，放入盆里，倒入适量的油和少量老抽腌制半小时，然后锅里倒适量油，中火将牛肉几面煎黄，如图。然后盛在盘子里。', '在等待腌制牛肉的时候，拍好大蒜和老姜，切好泰国绿辣椒', '将辅料准备好后，锅里倒油，花椒冷油下锅，小火熬五分钟，然后放入郫县豆瓣酱和干红辣椒，小火熬五分钟', '然后将八角，桂皮，小茴香，姜蒜倒入锅里，小火八分钟，再转中火四分钟，闻着香味出来了，转大火两分钟熬', '大火两分钟满后倒入牛肉，中火继续翻炒三分钟，加入没过牛肉的开水，记住开水下锅，牛肉不会柴。盖好锅盖，中火炖一个小时二十分钟，中间要开锅盖翻炒下牛肉，避免粘锅。', '然后将土豆去皮切中块，用蒸锅蒸十三分钟，用筷子插一下土豆基本可以插开就好，不用特别熟，一会儿后面还要继续下锅和牛肉一起烧', '将洋葱和葱切好备用，如图',"

In [18]:

formatted_messages = []
for msg in template_messages:
    formatted_content = msg.content.format(query=query, question=question)
    if isinstance(msg, SystemMessage):
        formatted_messages.append(SystemMessage(content=formatted_content))
    elif isinstance(msg, HumanMessage):
        formatted_messages.append(HumanMessage(content=formatted_content))

# 拼接成完整的 prompt 字符串（也可以直接传递消息列表给部分链）
full_prompt = "\n".join([msg.content for msg in formatted_messages])
full_prompt

'你是一个做菜小助手，名字叫智小厨，可以回答用户的问题。\n    - 你需要基于以下结构化菜谱知识推荐合适的菜谱，并给出推荐理由。\n    菜谱知识：\'在炖肉的同时，把土豆洗净去皮，切成滚刀块。胡萝卜切1cm左右的小丁。\', \'小葱洗净切成末，大蒜去皮切片。\', \'牛肉炖好后会发现肉汤上飘着一大层黑乎乎的沫子，这些都是牛肉里的脏东西。\', \'把牛肉一块块夹出，沥干水分置于小碗中。\', \'炒锅内倒少许油，待油热后放入蒜片炸出香味。\', \'倒入炖好的牛肉不停翻炒至肉香味飘出。\', \'倒入土豆块，继续不停翻炒至土豆略有粘锅。\', \'倒入没过食物的清水，加入盐、生抽、耗油、少许糖、料酒和老抽，并加入剩余的小米椒（不要掰段），倒入胡萝卜，转中火炖约20分钟。\', \'转大火，加入少许小葱末稍微煸炒几下，收汁出锅。\', \'如此美味，你抗拒的了吗？\']\nstep: [\'锅里烧水，水开后转为中火，倒入几滴食用油，将切好的牛肉块倒入锅里并倒入适量香米酒，飞水约七分钟，然后用热水将飞过水的牛肉洗净，提醒，一定要热水洗净，晾干，放入盆里，倒入适量的油和少量老抽腌制半小时，然后锅里倒适量油，中火将牛肉几面煎黄，如图。然后盛在盘子里。\', \'在等待腌制牛肉的时候，拍好大蒜和老姜，切好泰国绿辣椒\', \'将辅料准备好后，锅里倒油，花椒冷油下锅，小火熬五分钟，然后放入郫县豆瓣酱和干红辣椒，小火熬五分钟\', \'然后将八角，桂皮，小茴香，姜蒜倒入锅里，小火八分钟，再转中火四分钟，闻着香味出来了，转大火两分钟熬\', \'大火两分钟满后倒入牛肉，中火继续翻炒三分钟，加入没过牛肉的开水，记住开水下锅，牛肉不会柴。盖好锅盖，中火炖一个小时二十分钟，中间要开锅盖翻炒下牛肉，避免粘锅。\', \'然后将土豆去皮切中块，用蒸锅蒸十三分钟，用筷子插一下土豆基本可以插开就好，不用特别熟，一会儿后面还要继续下锅和牛肉一起烧\', \'将洋葱和葱切好备用，如图\',\n    你可以润色知识库的内容，但不能改变其含义。\n    你的回答需要以 JSON 格式字符串输出，包含以下字段：\n        - "recipe": 菜谱数据（从知识库中提取出菜品id返回）\n        - "reason": 推荐理由（字符串）\n    \n用户提问：土豆烧牛肉

In [19]:
# 调用对话链生成回答
for chunk in llm.stream(full_prompt):
    print(chunk, end="", flush=True)

```json
{
  "recipe": [
    {
      "id": "土豆烧牛肉"
    }
  ],
  "reason": "根据您的需求，推荐您尝试制作‘土豆烧牛肉’。这道菜以炖肉为主，搭配土豆和胡萝卜，口感丰富，营养均衡。同时，烹饪过程中需要注意去除肉中的杂质，使肉质更加鲜美。"
}
```

In [7]:
# 调用对话链生成回答
for chunk in llm.stream('你是谁'):
    print(chunk, end="", flush=True)

我是Qwen，由阿里云开发的大型语言模型。我旨在提供有用的信息和帮助，回答各种问题。有什么我可以帮你的吗？

In [8]:
# 调用对话链生成回答
for chunk in llm.stream('我上一个问题是什么'):
    print(chunk, end="", flush=True)

您上一个具体的问题是什么呢？由于对话历史的限制，我没有直接访问到具体的聊天记录。如果您能提供一些信息或上下文，我会更好地帮助您。之前我们讨论的内容可能涉及多个话题，包括但不限于语言模型、人工智能技术应用等。请告诉我您想了解的具体内容吧！